## Process earnings reports data from saved pickle

E. Quinn 12/20/2020

## Import standard python datascience packages

In [25]:
import sys
import math
import re
import copy
import numpy as np
import scipy as sc
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
%matplotlib inline

In [26]:
from datetime import datetime, timedelta, date
from datascience import *
import uuid
import random

### Import custom SC classes

In [27]:
sys.path.append("/home/gquinn/EG/school_committee/egsc/py_egsc")

In [28]:
from cba_salaries import Teacher_salaries 
from payperiod import Payperiod
from ucoa_labels import UCOA_labels
from eg_acct_codes import EG_acct_codes
from pay_check import Pay_check, Check_lineitem
from person import Person
from scenario import Scenario

In [29]:
ts = Teacher_salaries()
help(ts)

Help on Teacher_salaries in module cba_salaries object:

class Teacher_salaries(builtins.object)
 |  Methods defined here:
 |  
 |  __init__(self)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  decode_earnings(self, check_date, rate, in_earnings, obj)
 |  
 |  get_cba_matrix(self)
 |  
 |  get_future_code_from_code(self, code, incr)
 |      Returns CBA salary given step code for FY2013-FY2022.
 |  
 |  get_future_salary_from_code(self, code, incr)
 |      Returns CBA salary given step code for FY2013-FY2022.
 |  
 |  get_salary(self, fyear, step, col)
 |      Returns CBA salary given fiscal year, column, and step for FY2013-FY2022.
 |  
 |  get_salary_from_code(self, code)
 |      Returns CBA salary given step code for FY2013-FY2022.
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list o

### Show the directory we are running in

In [30]:
!pwd

/home/gquinn/EG/school_committee/finance_subcommittee/notebooks


### Load RIDE UCOA labels 

In [31]:
UCOA_labels = UCOA_labels()
    
#help(UCOA_labels)

### EG accounting codes class

provides descriptions for EG accounting codes and mapping to UCOA codes

In [32]:
EG_acct_codes = EG_acct_codes()
    
#help(EG_acct_codes)

### Load teacher salary matrix code

In [33]:
teacher_salary_matrix = Teacher_salaries()
    
#help(teacher_salary_matrix)

### Load serialized people object produced from earnings report

In [34]:
#Load data (deserialize)
with open('../../finance_subcommittee/people_12_22_2020.pkl', 'rb') as handle:
    people = pickle.load(handle)
    
len(people)

898

In [35]:
people

{'ELSON, SUSAN E': <person.Person at 0x7f34343fe280>,
 'RAKOVIC, EMILY E': <person.Person at 0x7f34345ae1f0>,
 'DEPASQUALE, HELEN L': <person.Person at 0x7f3434283820>,
 'MATTOX, JENNIFER L': <person.Person at 0x7f34342c1b20>,
 'ANDERSON, MARY C': <person.Person at 0x7f34341a3ca0>,
 'SMITH, MARY C': <person.Person at 0x7f3434095ee0>,
 'RINN, MICHELE H': <person.Person at 0x7f3433f27640>,
 'WILDENHAIN, DALE P': <person.Person at 0x7f3433dd34c0>,
 'KOWAL, MAUREEN E': <person.Person at 0x7f3433c57be0>,
 'GREENBERG, JUDITH A': <person.Person at 0x7f34339d6bb0>,
 'KENDALL, CYNTHIA A': <person.Person at 0x7f343374ba90>,
 'BUSHOVEN, LYNN S': <person.Person at 0x7f3433546dc0>,
 'TAYLOR, THERESA M': <person.Person at 0x7f34333ea070>,
 'DEGROOT, JUDITH Y': <person.Person at 0x7f3433302eb0>,
 'SIMONETTI, CATHERINE M': <person.Person at 0x7f34331c32b0>,
 'PLAZIAK, DAVID J': <person.Person at 0x7f3432f8f0a0>,
 'MATEER, PAMELA M': <person.Person at 0x7f3432fa7550>,
 'ALFANO, CAROL L': <person.Person

In [36]:
file1 = open('../earnings_detail.csv','w')

hstr = '"Name","School Year","Fiscal Year","Payperiod Number","Check Date","Check Number","Position","Fund",' \
    + '"Acct","Acct Description","Obj","Obj Description","Earnings",' \
    + '"Step","Payments","FTE","Salary"\n'
file1.write(hstr)

for name in people.keys():
    hstr = '"' + name + '","'
    pp = people[name].get_payperiods()
    for syr in pp.keys():
        hstr2 = hstr + syr + '",'
        for pseq in pp[syr].keys():
            fyear = pp[syr][pseq].get_fiscal_year()
            hstr3 = hstr2 + str(fyear) + ',' + str(pseq) + ','
            #hstr3 = hstr2
            cks = pp[syr][pseq].get_checks()
            for ckey in cks.keys():
                cdate = cks[ckey].get_date()
                cnumbr= cks[ckey].get_number()
                hstr4 = hstr3 + str(cdate) + ',' + str(cnumbr) + ','
                itms = cks[ckey].get_items()
                for itmkey in itms.keys():
                    fund = itms[itmkey].get_fund()
                    acct = itms[itmkey].get_acct()
                    obj  = itms[itmkey].get_obj()
                    pos  = itms[itmkey].get_position()
                    rate = itms[itmkey].get_rate()
                    accdesc = itms[itmkey].get_acct_desc()
                    objdesc = itms[itmkey].get_obj_desc()
                    earn    = itms[itmkey].get_earnings()
                    stepinfo = itms[itmkey].get_stepinfo()
                    if 'step' in stepinfo.keys():
                        step = stepinfo['step']
                    else:
                        step = ''
                    if 'payments' in stepinfo.keys():
                        pmts = stepinfo['payments']
                    else:
                        pmts = ''
                    if 'fte' in stepinfo.keys():
                        fte = stepinfo['fte']
                    else:
                        fte = ''
                    if 'salary' in stepinfo.keys():
                        sal = stepinfo['salary']
                    else:
                        sal = ''
                    hstr5 = hstr4 + pos + ',' + str(fund) + ',' + str(acct) + ',' + \
                    str(accdesc) + ',' + str(obj) + ',' + str(objdesc) + ',' + \
                    str(earn) + ',' + step + ',' + str(pmts) + ',' + str(fte) + ',' + \
                    str(sal) + '\n'
                    file1.write(hstr5)
file1.close()

In [24]:
p = people['ELSON, SUSAN E']
pp = p.get_payperiods()
for syear in pp.keys():
    for syseq in pp[syear].keys():
        po = pp[syear][syseq]
        pday = po.get_payday()
        chks = po.get_checks()
        print(pday,chks)
        for cnum in chks.keys():
            itms = chks[cnum].get_items()
            for itm in itms.keys():
                pcli = itms[itm]
                print(pday,cnum,itm,pcli.get_earnings())

2014-07-11 {'248396': <pay_check.Pay_check object at 0x7f3434e8ceb0>}
2014-07-11 248396 1 2967.85
2014-07-11 248396 2 2967.85
2014-07-11 248396 3 2967.85
2014-07-25 {'248415': <pay_check.Pay_check object at 0x7f3434e8ce50>}
2014-07-25 248415 1 2967.85
2014-07-25 248415 2 2967.85
2014-07-25 248415 3 2967.85
2014-08-08 {'248434': <pay_check.Pay_check object at 0x7f34345ee490>}
2014-08-08 248434 1 2967.85
2014-08-08 248434 2 2967.85
2014-08-08 248434 3 2967.85
2014-08-22 {'249658': <pay_check.Pay_check object at 0x7f34345ee040>}
2014-08-22 249658 1 3027.19
2014-08-22 249658 2 3027.19
2014-08-22 249658 3 3027.19
2014-09-05 {'249827': <pay_check.Pay_check object at 0x7f34345ee430>}
2014-09-05 249827 1 3027.19
2014-09-05 249827 2 3027.19
2014-09-05 249827 3 3027.19
2014-09-19 {'250036': <pay_check.Pay_check object at 0x7f34345ee2b0>}
2014-09-19 250036 1 3027.19
2014-09-19 250036 2 3027.19
2014-09-19 250036 3 3027.19
2014-10-03 {'250252': <pay_check.Pay_check object at 0x7f34345ee790>}
2014-1